## Root Finding (and Fixed Points)

There are many many examples from economics we seek to either find the root or the fixed point to a non-linear of (often many) equations, which cannot be computed analytically.

Many estimation algorithms for equilibrium problems involve a nested structure where there is some root-finding problem in the inner nest: BLP, dynamic discrete choice (rust, labor models), trade models, steady state of dynamic models.


*Defn:* Roots.
A function $g$ from $R^n$ to $R^n$ is given and one must find a vector $x$ that satisfies $x = g(x)$.

<img src="files/graph1.png" width="75%"/>


#### Examples

- Demand and Supply market clearing.
- FOCs from an optimization problem.


### Iterative Methods

We will consider methods that *systematically* look over the range of $x$ until $f(x) = 0$

### Bisection Method

Intermediate Value Theorem: If a continuous real-valued function assumes two distinct values, then it must assume all values in between. 

If $f$ is continuous and $f(a)$ and $f(b)$ have different signs, then there must be at least one root $x$ in $[a,b]$.

Evaluate $f$ at the bisection of $a$ and $b$. Take the new interval to be the bisected interval with endpoints of different signs. Repeat. 

<img src="files/graph1.png" width="75%"/>

#### Bisection Example

In [14]:
# Set up packages
using Printf


In [15]:
f(x) = x.^3

f (generic function with 1 method)

In [16]:
a = -6.0;
b = 12.0;

In [17]:
tol = 1e-4;
s = sign(f(a)); # sign if the left boundary
x = (a+b)/2; # inital midpoint
d = (b-a)/2;
xsave=[];

In [20]:
while d>tol
	d=d/2; # length to cut the next interval
	push!(xsave, x)
	if s == sign(f(x))
		x = x+d;
	else
		x = x-d;
	end
end
push!(xsave,x)

@printf "Solution using user written code: x = %3.5f" x

Solution using user written code: x = -0.00002

**Pros and cons of bisection**

* Pro: Guaranteed to find a root.
* Con: Slow (no gradient information).
* Con: Will only find one root.
* Con: Only good for single variable functions
* Con: Can be very slow b/c it does not use info on shape of function

### Function iteration

* Supply a guess $x^0$
* Use the updating rule $x^{(t+1)} \leftarrow g(x^(t))$.
 
The starting guess must be close to the fixed point where $||g'(x*)||<1$


<br><br><br><br>
<img src="files/graph2.png" width="75%"/>
<br><br><br><br>


#### Example



In [26]:

function fncIteration(g::Function,x::Float64)

    d = 100.0
    x = 0.1

    while d>tol
    
        d = abs(x-g(x))
        x = g(x)
    
    end

    return x
end

g(x) = x.^0.5;
x = 0.1

sol = fncIteration(g,x)

@printf "Solution using user written code: x = %3.5f" sol

Solution using user written code: x = 0.99993


<br><br><br><br>
<video width="640" height="480" controls>
<source src="testmovie-below.mp4" type="video/mp4">
</video>
<br><br><br><br>


### Newton's Method

* Use derivative information
* Probably most common method.
* Sometimes we know the derivative (pen a paper).
* Sometimes we need to approximate the derivative.
* Same thing goes with second derivatives. 

*The idea:*

1. guess a point
2. linearize the function around that point 
3. find the root of the linear function using taylor expansion
4. use that point as your new guess and repeat



<br><br><br><br>
<img src="files/graph3.png" width="25%"/>
<br><br><br><br>




First-order Taylor approximation: $f(x)\approx f(x^t) + f'(x^t)(x - x^t) = 0$

which yields the following iteration rule: $x^{t+1}\leftarrow x^t - [f'(x^t)]^{-1}f(x^t)$

* *What do you notice about this iterative method?*
* We need to know the derivative!
* We will discuss this in detail later.


*Convergence*: Judd Theorem 2.1 (page 130) -- If $x^1$ is "sufficiently" close
to $x^*$, $f'(x^*)\ne0$ and $\mid \frac{f''(x^*)}{f'(x^*)}<\infty$, then the Newton
sequence will converge to $x^*$. Also, $f$ needs to be "smooth."

* Warning: if $f'(x^t)$ is close to zero, then it can overshoot and cause problems



#### Newton Example

Simple demand function in a separate file:


In [30]:
# Demand
q(p) = -12 + 2*p.^(-3);

# 1st derivative of demand
Dq(p) = -6*p.^(-4);


In [34]:
function nm(f::Function, fp::Function, x::Float64; tol=0.0001::Float64)
   
    ctr, max_steps = 0, 100
     
    while (abs(f(x)) > tol) && ctr < max_steps
        x = x - f(x) / fp(x)
        ctr = ctr + 1
    end

    ctr >= max_steps ? error("Method did not converge") : return (x, ctr)
    
end



nm (generic function with 2 methods)

In [35]:
sol_nm = nm(q,Dq,0.1)

@printf "Solution using user written code: x = %3.5f" sol_nm

Solution using user written code: x = 

LoadError: [91mMethodError: no method matching isfinite(::Tuple{Float64,Int64})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  isfinite([91m::BigFloat[39m) at mpfr.jl:885[39m
[91m[0m  isfinite([91m::Missing[39m) at missing.jl:100[39m
[91m[0m  isfinite([91m::Float16[39m) at float.jl:555[39m
[91m[0m  ...[39m